海量数据处理常用数据结构

https://www.cnblogs.com/shawshawwan/p/9500952.html

## 数据结构：

【Bloom Filter】

  它实际上是一个很长的二进制向量和一系列随机映射函数

  布隆过滤器可以用于检索一个元素是否在一个集合中

  它的优点是空间效率和查询时间都远远超过一般的算法，缺点是有一定的误识别率和删除困难

 

【Bit map】

  Bit-map就是用一个bit位来标记某个元素对应的Value， 而Key即是该元素。由于采用了Bit为单位来存储数据，因此在存储空间方面，可以大大节省

  

 

【Hash】

【Trie】

【堆】


## 常见思想：

【分而治之/hash映射 + hash统计 + 堆/快速/归并排序】

 对大文件使用hash取模映射成小文件，再在小文件中分别统计

 

【双层桶划分】

【Trie树/数据库/倒排索引】

【外排序】

【分布式处理之Hadoop/Mapreduce】


## 面试题剖析

海量数据面试题的常见考点，无非就是两个，一是数据太大，无法一次性装入内存；二是数据量太大，无法单机快速处理

常用的思想就是分而治之，先化大为小，化繁为简，再归并结果

 

**例1:  海量日志数据，提取出某日访问百度次数最多的那个IP**

（文件总量多大 -> 能一次载入内存吗 -> 怎么将文件化大为小，一般可以采取hash -> 然后怎么归并）

首先可以估计下某天全部的日志大小，当然可以选取任一个合理的估计 ，而不需要很精确。比如IP总的可能情况是？

解答思路：  
1.IP地址最多有2^32=4G种取值情况，所以不能完全加载到内存中处理；  
2.可以考虑采用“分而治之”的思想，按照IP地址的Hash(IP)%1024值，把海量IP日志分别存储到1024个小文件中。这样，每个小文件最多包含4MB个IP地址；   
3.对于每一个小文件，可以构建一个IP为key，出现次数为value的Hash map，同时记录当前出现次数最多的那个IP地址；  
4.可以得到1024个小文件中的出现次数最多的IP，再依据常规的排序算法得到总体上出现次数最多的IP


**例2: 搜索引擎会通过日志文件把用户每次检索使用的所有检索串都记录下来，每个查询串的长度为1-255字节。
假设目前有一千万个记录（这些查询串的重复度比较高，虽然总数是1千万，但如果除去重复后，不超过3百万个。一个查询串的重复度越高，说明查询它的用户越多，也就是越热门），请你统计最热门的10个查询串，要求使用的内存不能超过1G**


1.hash_map统计：先对这批海量数据预处理。具体方法是：维护一个Key为Query字串，Value为该Query出现次数的HashTable，即hash_map(Query，Value)，每次读取一个Query，如果该字串不在Table中，那么加入该字串，并且将Value值设为1；如果该字串在Table中，那么将该字串的计数加一即可。最终我们在O(N)的时间复杂度内用Hash表完成了统计；

2.堆排序：第二步、借助堆这个数据结构，找出Top K，时间复杂度为N‘logK。即借助堆结构，我们可以在log量级的时间内查找和调整/移动。因此，维护一个K(该题目中是10)大小的小根堆，然后遍历300万的Query，分别和根元素进行对比。所以，我们最终的时间复杂度是：O（N） + N' * O（logK），（N为1000万，N’为300万）


**例3: 有一个1G大小的一个文件，里面每一行是一个词，词的大小不超过16字节，内存限制大小是1M。返回频数最高的100个词**
分而治之 + hash统计 + 堆/快速排序

1.分而治之/hash映射：顺序读文件中，对于每个词x，取hash(x)%5000，然后按照该值存到5000个小文件（记为x0,x1,...x4999）中。这样每个文件大概是200k左右。如果其中的有的文件超过了1M大小，还可以按照类似的方法继续往下分，直到分解得到的小文件的大小都不超过1M。  
2.hash_map统计：对每个小文件，采用trie树/hash_map等统计每个文件中出现的词以及相应的频率。  
3.堆/归并排序：取出出现频率最大的100个词（可以用含100个结点的最小堆）后，再把100个词及相应的频率存入文件，这样又得到了5000个文件。最后就是把这5000个文件进行归并（类似于归并排序）的过程了。  